# CONTROL DE HONORARIOS (partida 300)

La finalidad del presente reporte es cruzar la información de ejecución de honorarios factureros (Slave) y el Resumen de Rendiciones por Proveedor del SGF. En la medida que la información de Slave haya sido previamente cruzada con SIIF, este control permitirá detectar, en mayor medida, desvíos en la carga del SIIF. 
**Tener en cuenta que, por el momenos, no es posible filtrar el los pagos con destino reparto de chequera del SGF**. Dado que los mismos son cargados en Slave, esto genera un importante **desvío a la hora del control**.

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [ ]:
from invicoctrlpy.gastos.control_honorarios.control_honorarios import ControlHonorarios

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment',None)

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

from  pivottablejs import pivot_ui

EJERCICIO = ['2023']
UPDATE_DB = False
control_honorarios = ControlHonorarios(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Controles

### Control Carga Slave vs SIIF

In [ ]:
siif_vs_slave = control_honorarios.siif_vs_slave()
itables.show(siif_vs_slave, column_filters="footer", dom="lrtip")

### Control por Mes y Cuenta Corriente

El presente reporte agrupa la información del Slave y el SGF por mes y cuenta corriente para luego cruzarlos. En las últimas columnas se exponen las diferencias detectas entre ambos sistemas en relación al pago de honorarios de forma mensual y acumulada. 

In [ ]:
slave_vs_sgf = control_honorarios.slave_vs_sgf(only_diff=True)
itables.show(slave_vs_sgf, column_filters="footer", dom="lrtip")

## Tablas Auxiliares

### Slave

Listado de comprobantes SIIF registrados por SLAVE. Se adiciona el campo cuenta corriente del reporte rcg01_uejp (SIIF). Queda pendiente, para próximas actualizaciones, netear el impacto de la Deuda Flotante (RDEU)

In [ ]:
slave = control_honorarios.import_slave()
itables.show(slave, column_filters="footer", dom="lrtip")

### Resumen de Rendiciones por Proveedores (SGF)

El listado que a continuación se detalla corresponde a los registros de la Resumen de Rendiciones por Proveedor del SGF filtrado de a acuerdo a los siguientes parámetros:
 - no se incluye el origen OBRAS
 - cuenta corriente incluidas 130832-05 y 130832-07
 - destinos incluidos: 'HONORARIOS - FUNCIONAMIENTO', 'COMISIONES - FUNCIONAMIENTO', 'HONORARIOS - EPAM'

Además, se incluyen los Depósitos de Embargos por Alimiento retendidos a los factureros de la cuenta 130832-05 en base a registros del Sistema de Seguimiento de Cuentas Corrientes.

**Queda pendiente el neteo del concepto Distribución de Chequeras** (forman parte del SGF pero no de Slave)

In [ ]:
sgf = control_honorarios.import_resumen_rend_honorarios()
itables.show(sgf, column_filters="footer", dom="lrtip")

### SIIF

In [ ]:
siif = control_honorarios.import_siif_comprobantes()
itables.show(siif, column_filters="footer", dom="lrtip")

## EXTRA:

### Excel

In [ ]:
with pd.ExcelWriter('control_honorarios.xlsx') as writer:
    siif_vs_slave.to_excel(writer, sheet_name='siif_vs_slave', index=False)
    slave_vs_sgf.to_excel(writer, sheet_name='slave_vs_sgf', index=False)
    slave.to_excel(writer, sheet_name='slave', index=False)
    siif.to_excel(writer, sheet_name='siif', index=False)
    sgf.to_excel(writer, sheet_name='sgf', index=False)